In [1]:
import sys
sys.path.append('../')
import networkx as nx
import time
from smodels.theory.exceptions import SModelSTheoryError as SModelSError
from smodels.tools.smodelsLogging import logger
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from smodels.theory.element import Element
from smodels.theory.topology import TopologyList
from smodels.theory.crossSection import XSection,XSectionInfo,XSectionList
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.physicsUnits import fb, GeV
from smodels.theory.tree import Tree,ParticleNode
from smodels.theory.decomposer import cascadeDecay, addOneStepDecays
import itertools
%load_ext line_profiler

### Decomposer

In [2]:
def decompose(model, sigmacut= 0*fb, doCompress=True, doInvisible=True,
              minmassgap= 0*GeV):
    """
    Perform decomposition using the information stored in model.
    
    :param sigmacut: minimum sigma*BR to be generated, by default sigmacut = 0.1 fb
    :param doCompress: turn mass compression on/off
    :param doInvisible: turn invisible compression on/off
    :param minmassgap: maximum value (in GeV) for considering two R-odd particles
                       degenerate (only revelant for doCompress=True )
    :returns: list of topologies (TopologyList object)

    """
    t1 = time.time()
    
    xSectionList = model.xsections    
    pdgList = model.getValuesFor('pdg')

    if doCompress and minmassgap/GeV < 0.:
        logger.error("Asked for compression without specifying minmassgap. Please set minmassgap.")        
        raise SModelSError()

    if isinstance(sigmacut,(float,int)):
        sigmacut = float(sigmacut) * fb

    xSectionList.removeLowerOrder()
    # Order xsections by highest xsec value to improve performance
    xSectionList.sort()

    # Generate all primary nodes (e.g. PV > X+Y)
    # and assign the nodeWeight as the maximum cross-section
    productionTrees = []
    for pid in xSectionList.getPIDpairs():
        weight = xSectionList.getXsecsFor(pid)
        if weight < sigmacut:
            continue
        pv = ParticleNode(model.getParticlesWith(label='PV')[0],0,nodeWeight=weight)
        pv.xsection = xSectionList.getXsecsFor(pid)
        primaryMothers = [ParticleNode(model.getParticlesWith(pdg=pdg)[0],i+1) for i,pdg in enumerate(pid)]
        productionTrees.append(Tree({pv : primaryMothers}))

    # Sort production trees
    productionTrees = sorted(productionTrees, key = lambda t: t.getTreeWeight().getMaxXsec(), reverse=True)
    
    
    print('%i production trees' %len(productionTrees))
#     return productionTrees
    # For each production tree, produce all allowed cascade decays (above sigmacut):
    allTrees = []
    for tree in productionTrees:
#         print('len=',len(allTrees))
        allTrees += cascadeDecay(tree,sigmacut=sigmacut)

    print('%i decayed trees' %len(allTrees))
    return allTrees

    # Create elements for each tree and combine equal elements
    smsTopList = TopologyList()

    for tree in allTrees:                                       
        newElement = Element(tree)
        newElement.weight = tree.getTreeWeight()
        smsTopList.addElement(newElement)                                                 
                                                    
    smsTopList.compressElements(doCompress, doInvisible, minmassgap)
    smsTopList._setElementIds()    
    
    print("decomposer done in %.2f s." % (time.time() -t1 ) )
    
    return smsTopList

### Load model

In [3]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
# slhafile = '../inputFiles/slha/simplyGluino.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


In [4]:
sigmacut = 0.1*fb
trees = decompose(model, sigmacut= sigmacut)
nTotal = 14647
nUnique = 5624
print('(expected = %i trees)' %nTotal)

43 production trees
14647 decayed trees
(expected = 14647 trees)


In [5]:
%lprun -f addOneStepDecays decompose(model, sigmacut= sigmacut)

43 production trees
14647 decayed trees
